###### > # **Part - I**

*   define actor critic networks
*   define PPO algorithm

################################################################################

In [5]:


############################### Import libraries ###############################


import os
import glob
import time
from datetime import datetime

import torch
import torch.nn as nn
from torch.distributions import MultivariateNormal
from torch.distributions import Categorical

import numpy as np

import gym
import pongGym

################################## set device ##################################

print("============================================================================================")


# set device to cpu or cuda
device = torch.device('cpu')

if(torch.cuda.is_available()): 
    device = torch.device('cuda:0') 
    torch.cuda.empty_cache()
    print("Device set to : " + str(torch.cuda.get_device_name(device)))
else:
    print("Device set to : cpu")
    
print("============================================================================================")




################################## PPO Policy ##################################


class RolloutBuffer:
    def __init__(self):
        self.actions = []
        self.states = []
        self.logprobs = []
        self.rewards = []
        self.is_terminals = []
    

    def clear(self):
        del self.actions[:]
        del self.states[:]
        del self.logprobs[:]
        del self.rewards[:]
        del self.is_terminals[:]


class ActorCritic(nn.Module):
    def __init__(self, state_dim, action_dim, action_std_init):
        super(ActorCritic, self).__init__()

#         self.quant = torch.quantization.QuantStub()
        self.actor = nn.Sequential(
                        torch.quantization.QuantStub(),
                        nn.Linear(state_dim, 64),
                        nn.ReLU(),
                        nn.Linear(64, 64),
                        nn.ReLU(),
                        nn.Linear(64, 64), 
                        nn.ReLU(),
                        nn.Linear(64, 64), 
                        nn.ReLU(),
                        nn.Linear(64, action_dim),
                        torch.quantization.DeQuantStub(),
                        nn.Softmax(dim=-1)
                    )
#         self.dequant = torch.quantization.DeQuantStub()
        # critic
        self.critic = nn.Sequential(
                        nn.Linear(state_dim, 64),
                        nn.ReLU(),
                        nn.Linear(64, 64),
                        nn.ReLU(),
                        nn.Linear(64, 64),
                        nn.ReLU(),
                        nn.Linear(64, 64), 
                        nn.ReLU(),
                        nn.Linear(64, 1)
                    )
    def forward(self):
        raise NotImplementedError
    

    def act(self, state):

        
        action_probs = self.actor(state)
        dist = Categorical(action_probs)

        action = dist.sample()
        action_logprob = dist.log_prob(action)
        
        return action.detach(), action_logprob.detach()
    

    def evaluate(self, state, action):


        action_probs = self.actor(state)
        dist = Categorical(action_probs)

        action_logprobs = dist.log_prob(action)
        dist_entropy = dist.entropy()
        state_values = self.critic(state)
        
        return action_logprobs, state_values, dist_entropy


class PPO:
    def __init__(self, state_dim, action_dim, lr_actor, lr_critic, gamma, K_epochs, eps_clip, action_std_init=0.6):

        self.gamma = gamma
        self.eps_clip = eps_clip
        self.K_epochs = K_epochs
        
        self.buffer = RolloutBuffer()

        self.policy = ActorCritic(state_dim, action_dim, action_std_init).to(device)
        self.optimizer = torch.optim.Adam([
                        {'params': self.policy.actor.parameters(), 'lr': lr_actor},
                        {'params': self.policy.critic.parameters(), 'lr': lr_critic}
                    ])

        self.policy_old = ActorCritic(state_dim, action_dim, action_std_init).to(device)
        self.policy_old.load_state_dict(self.policy.state_dict())
        
        self.MseLoss = nn.MSELoss()


    def select_action(self, state):

        with torch.no_grad():
            state = torch.FloatTensor(state).to(device)
            action, action_logprob = self.policy_old.act(state)

        self.buffer.states.append(state)
        self.buffer.actions.append(action)
        self.buffer.logprobs.append(action_logprob)

        return action.item()


    def update(self):

        # Monte Carlo estimate of returns
        rewards = []
        discounted_reward = 0
        for reward, is_terminal in zip(reversed(self.buffer.rewards), reversed(self.buffer.is_terminals)):
            if is_terminal:
                discounted_reward = 0
            discounted_reward = reward + (self.gamma * discounted_reward)
            rewards.insert(0, discounted_reward)
            
        # Normalizing the rewards
        rewards = torch.tensor(rewards, dtype=torch.float32).to(device)
        rewards = (rewards - rewards.mean()) / (rewards.std() + 1e-7)

        # convert list to tensor
        old_states = torch.squeeze(torch.stack(self.buffer.states, dim=0)).detach().to(device)
        old_actions = torch.squeeze(torch.stack(self.buffer.actions, dim=0)).detach().to(device)
        old_logprobs = torch.squeeze(torch.stack(self.buffer.logprobs, dim=0)).detach().to(device)

        
        # Optimize policy for K epochs
        for _ in range(self.K_epochs):

            # Evaluating old actions and values
            logprobs, state_values, dist_entropy = self.policy.evaluate(old_states, old_actions)

            # match state_values tensor dimensions with rewards tensor
            state_values = torch.squeeze(state_values)
            
            # Finding the ratio (pi_theta / pi_theta__old)
            ratios = torch.exp(logprobs - old_logprobs.detach())

            # Finding Surrogate Loss
            advantages = rewards - state_values.detach()   
            surr1 = ratios * advantages
            surr2 = torch.clamp(ratios, 1-self.eps_clip, 1+self.eps_clip) * advantages

            # final loss of clipped objective PPO
            loss = -torch.min(surr1, surr2) + 0.5*self.MseLoss(state_values, rewards) - 0.01*dist_entropy
            
            # take gradient step
            self.optimizer.zero_grad()
            loss.mean().backward()
            self.optimizer.step()
            
        # Copy new weights into old policy
        self.policy_old.load_state_dict(self.policy.state_dict())

        # clear buffer
        self.buffer.clear()
    
    
    def save(self, checkpoint_path):
        torch.save(self.policy_old.state_dict(), checkpoint_path)
   

    def load(self, checkpoint_path):
        self.policy_old.load_state_dict(torch.load(checkpoint_path, map_location=lambda storage, loc: storage))
        self.policy.load_state_dict(torch.load(checkpoint_path, map_location=lambda storage, loc: storage))
        
        
       



Device set to : NVIDIA GeForce RTX 3070


################################################################################
> # **Part - II**

*   train PPO algorithm on environments
*   save preTrained networks weights and log files

################################################################################

In [ ]:

print("============================================================================================")


################################### Training ###################################


####### initialize environment hyperparameters ######

env_name = "pong_game"

max_ep_len = 60*120                    # max timesteps in one episode
max_training_timesteps = int(1e5)   # break training loop if timeteps > max_training_timesteps

print_freq = max_ep_len * 3     # print avg reward in the interval (in num timesteps)
log_freq = max_ep_len * 2       # log avg reward in the interval (in num timesteps)
save_model_freq = int(2e4)      # save model frequency (in num timesteps)

action_std = None


#####################################################


## Note : print/log frequencies should be > than max_ep_len


################ PPO hyperparameters ################


update_timestep = max_ep_len * 10      # update policy every n timesteps
K_epochs = 60*10               # update policy for K epochs
# K_epochs = 4
eps_clip = 0.2              # clip parameter for PPO
gamma = 0.99                # discount factor

lr_actor = 0.0001       # learning rate for actor network
lr_critic = 0.001       # learning rate for critic network

random_seed = 910426         # set random seed if required (0 = no random seed)

#####################################################



print("training environment name : " + env_name)

env = pongGym.DoublePong()

# state space dimension
state_dim = env.observation_space.shape[0]

# action space dimension

action_dim = env.action_space.n



###################### logging ######################

#### log files for multiple runs are NOT overwritten

log_dir = "PPO_logs"
if not os.path.exists(log_dir):
      os.makedirs(log_dir)

log_dir = log_dir + '/' + env_name + '/'
if not os.path.exists(log_dir):
      os.makedirs(log_dir)


#### get number of log files in log directory
run_num = 0
current_num_files = next(os.walk(log_dir))[2]
run_num = len(current_num_files)


#### create new log file for each run 
log_f_name = log_dir + '/PPO_' + env_name + "_log_" + str(run_num) + ".csv"

print("current logging run number for " + env_name + " : ", run_num)
print("logging at : " + log_f_name)

#####################################################


################### checkpointing ###################

run_num_pretrained = 1      #### change this to prevent overwriting weights in same env_name folder

directory = "PPO_preTrained"
if not os.path.exists(directory):
      os.makedirs(directory)

directory = directory + '/' + env_name + '/'
if not os.path.exists(directory):
      os.makedirs(directory)

#####################################################


############# print all hyperparameters #############

print("--------------------------------------------------------------------------------------------")

print("max training timesteps : ", max_training_timesteps)
print("max timesteps per episode : ", max_ep_len)

print("model saving frequency : " + str(save_model_freq) + " timesteps")
print("log frequency : " + str(log_freq) + " timesteps")
print("printing average reward over episodes in last : " + str(print_freq) + " timesteps")

print("--------------------------------------------------------------------------------------------")

print("state space dimension : ", state_dim)
print("action space dimension : ", action_dim)

print("--------------------------------------------------------------------------------------------")

print("Initializing a discrete action space policy")

print("--------------------------------------------------------------------------------------------")

print("PPO update frequency : " + str(update_timestep) + " timesteps") 
print("PPO K epochs : ", K_epochs)
print("PPO epsilon clip : ", eps_clip)
print("discount factor (gamma) : ", gamma)

print("--------------------------------------------------------------------------------------------")

print("optimizer learning rate actor : ", lr_actor)
print("optimizer learning rate critic : ", lr_critic)

if random_seed:
    print("--------------------------------------------------------------------------------------------")
    print("setting random seed to ", random_seed)
    torch.manual_seed(random_seed)
#     env.seed(random_seed)
    np.random.seed(random_seed)

#####################################################

print("============================================================================================")

################# training procedure ################

# initialize a PPO agent
ppo_agent = PPO(state_dim, action_dim, lr_actor, lr_critic, gamma, K_epochs, eps_clip, action_std)


# track total training time
start_time = datetime.now().replace(microsecond=0)
print("Started training at (GMT) : ", start_time)

print("============================================================================================")


# logging file
log_f = open(log_f_name,"w+")
log_f.write('episode,timestep,reward\n')


# printing and logging variables
print_running_reward = 0
print_running_episodes = 0

log_running_reward = 0
log_running_episodes = 0

time_step = 0
i_episode = 0

Max_reward = 0
Max_episode = 0
# training loop
# env = pongGym.DoublePong()
while i_episode < 100000:
    
    state = env.reset()
    current_ep_reward = 0
    for t in range(1, max_ep_len+1):
        
        # select action with policy
        action = ppo_agent.select_action(state)
        state, reward, done, _ = env.step(action)
#         if i_episode > 100:
#             env.render()
        
        # saving reward and is_terminals
        ppo_agent.buffer.rewards.append(reward)
        ppo_agent.buffer.is_terminals.append(done)
        
        time_step +=1
        current_ep_reward += reward

        # update PPO agent
        if time_step % update_timestep == 0:
            ppo_agent.update()

        # log in logging file
        if time_step % log_freq == 0:

            # log average reward till last episode
            log_avg_reward = log_running_reward / log_running_episodes
            log_avg_reward = round(log_avg_reward, 4)

            log_f.write('{},{},{}\n'.format(i_episode, time_step, log_avg_reward))
            log_f.flush()

            log_running_reward = 0
            log_running_episodes = 0
            
        # printing average reward
        if time_step % print_freq == 0:

            # print average reward till last episode
            print_avg_reward = print_running_reward / print_running_episodes
            print_avg_reward = round(print_avg_reward, 2)

            print("Episode : {} \t\t Timestep : {} \t\t Average Reward : {}".format(i_episode, time_step, print_avg_reward))
            print_running_reward = 0
            print_running_episodes = 0
        # break; if the episode is over
        if done:
            break

    # save model weights
#     print("Episode : {} \t\t Totol reward : {}".format(i_episode, current_ep_reward))
    if i_episode >= Max_episode + 1000:
        Max_episode = i_episode
        checkpoint_path = directory + "PPO_{}_{}_{}.pth".format(env_name, int(Max_episode), run_num_pretrained)
        print("--------------------------------------------------------------------------------------------")
        print("saving model at : " + checkpoint_path)
        ppo_agent.save(checkpoint_path)
        print("model saved")
        print("Elapsed Time  : ", datetime.now().replace(microsecond=0) - start_time)
        print("--------------------------------------------------------------------------------------------")
    print_running_reward += current_ep_reward
    print_running_episodes += 1

    log_running_reward += current_ep_reward
    log_running_episodes += 1

    i_episode += 1


log_f.close()
env.close()




# print total training time
print("============================================================================================")
end_time = datetime.now().replace(microsecond=0)
print("Started training at (GMT) : ", start_time)
print("Finished training at (GMT) : ", end_time)
print("Total training time  : ", end_time - start_time)
print("============================================================================================")







training environment name : pong_game
current logging run number for pong_game :  1
logging at : PPO_logs/pong_game//PPO_pong_game_log_1.csv
--------------------------------------------------------------------------------------------
max training timesteps :  100000
max timesteps per episode :  7200
model saving frequency : 20000 timesteps
log frequency : 14400 timesteps
printing average reward over episodes in last : 21600 timesteps
--------------------------------------------------------------------------------------------
state space dimension :  24
action space dimension :  4
--------------------------------------------------------------------------------------------
Initializing a discrete action space policy
--------------------------------------------------------------------------------------------
PPO update frequency : 72000 timesteps
PPO K epochs :  600
PPO epsilon clip :  0.2
discount factor (gamma) :  0.99
--------------------------------------------------------------------

Episode : 326 		 Timestep : 2354400 		 Average Reward : 75.76
Episode : 329 		 Timestep : 2376000 		 Average Reward : 141.45
Episode : 332 		 Timestep : 2397600 		 Average Reward : 129.46
Episode : 335 		 Timestep : 2419200 		 Average Reward : 40.73
Episode : 338 		 Timestep : 2440800 		 Average Reward : 148.77
Episode : 341 		 Timestep : 2462400 		 Average Reward : 133.83
Episode : 344 		 Timestep : 2484000 		 Average Reward : 93.96
Episode : 347 		 Timestep : 2505600 		 Average Reward : 51.13
Episode : 350 		 Timestep : 2527200 		 Average Reward : 63.3
Episode : 353 		 Timestep : 2548800 		 Average Reward : 58.4
Episode : 356 		 Timestep : 2570400 		 Average Reward : 75.6
Episode : 359 		 Timestep : 2592000 		 Average Reward : 81.21
Episode : 362 		 Timestep : 2613600 		 Average Reward : 57.45
Episode : 365 		 Timestep : 2635200 		 Average Reward : 82.37
Episode : 368 		 Timestep : 2656800 		 Average Reward : 139.51
Episode : 371 		 Timestep : 2678400 		 Average Reward : 49.99
Episod

Episode : 722 		 Timestep : 5205600 		 Average Reward : 73.02
Episode : 725 		 Timestep : 5227200 		 Average Reward : -56.69
Episode : 728 		 Timestep : 5248800 		 Average Reward : 128.18
Episode : 731 		 Timestep : 5270400 		 Average Reward : 237.11
Episode : 734 		 Timestep : 5292000 		 Average Reward : 123.0
Episode : 737 		 Timestep : 5313600 		 Average Reward : 296.93
Episode : 740 		 Timestep : 5335200 		 Average Reward : 225.63
Episode : 743 		 Timestep : 5356800 		 Average Reward : 111.48
Episode : 746 		 Timestep : 5378400 		 Average Reward : 279.94
Episode : 749 		 Timestep : 5400000 		 Average Reward : 134.12
Episode : 752 		 Timestep : 5421600 		 Average Reward : 264.96
Episode : 755 		 Timestep : 5443200 		 Average Reward : 156.47
Episode : 758 		 Timestep : 5464800 		 Average Reward : 226.11
Episode : 761 		 Timestep : 5486400 		 Average Reward : 72.31
Episode : 764 		 Timestep : 5508000 		 Average Reward : 326.16
Episode : 767 		 Timestep : 5529600 		 Average Reward : 13

Episode : 1100 		 Timestep : 7927200 		 Average Reward : 295.67
Episode : 1103 		 Timestep : 7948800 		 Average Reward : 157.19
Episode : 1106 		 Timestep : 7970400 		 Average Reward : 292.64
Episode : 1109 		 Timestep : 7992000 		 Average Reward : 254.5
Episode : 1112 		 Timestep : 8013600 		 Average Reward : 141.27
Episode : 1115 		 Timestep : 8035200 		 Average Reward : 240.94
Episode : 1118 		 Timestep : 8056800 		 Average Reward : 161.49
Episode : 1121 		 Timestep : 8078400 		 Average Reward : 194.73
Episode : 1124 		 Timestep : 8100000 		 Average Reward : 169.62
Episode : 1127 		 Timestep : 8121600 		 Average Reward : 295.61
Episode : 1130 		 Timestep : 8143200 		 Average Reward : 199.95
Episode : 1133 		 Timestep : 8164800 		 Average Reward : 217.35
Episode : 1136 		 Timestep : 8186400 		 Average Reward : 164.03
Episode : 1139 		 Timestep : 8208000 		 Average Reward : 260.24
Episode : 1142 		 Timestep : 8229600 		 Average Reward : 329.21
Episode : 1145 		 Timestep : 8251200 		 A

Episode : 1484 		 Timestep : 10692000 		 Average Reward : 211.41
Episode : 1487 		 Timestep : 10713600 		 Average Reward : 325.74
Episode : 1490 		 Timestep : 10735200 		 Average Reward : 424.56
Episode : 1493 		 Timestep : 10756800 		 Average Reward : 358.94
Episode : 1496 		 Timestep : 10778400 		 Average Reward : 325.05
Episode : 1499 		 Timestep : 10800000 		 Average Reward : 282.88
Episode : 1502 		 Timestep : 10821600 		 Average Reward : 267.02
Episode : 1505 		 Timestep : 10843200 		 Average Reward : 439.16
Episode : 1508 		 Timestep : 10864800 		 Average Reward : 345.09
Episode : 1511 		 Timestep : 10886400 		 Average Reward : 419.05
Episode : 1514 		 Timestep : 10908000 		 Average Reward : 457.33
Episode : 1517 		 Timestep : 10929600 		 Average Reward : 389.27
Episode : 1520 		 Timestep : 10951200 		 Average Reward : 331.32
Episode : 1523 		 Timestep : 10972800 		 Average Reward : 229.29
Episode : 1526 		 Timestep : 10994400 		 Average Reward : 299.87
Episode : 1529 		 Timeste

...

Episode : 14174 		 Timestep : 102060000 		 Average Reward : 525.3
Episode : 14177 		 Timestep : 102081600 		 Average Reward : 434.61
Episode : 14180 		 Timestep : 102103200 		 Average Reward : 451.91
Episode : 14183 		 Timestep : 102124800 		 Average Reward : 642.18
Episode : 14186 		 Timestep : 102146400 		 Average Reward : 465.48
Episode : 14189 		 Timestep : 102168000 		 Average Reward : 404.42
Episode : 14192 		 Timestep : 102189600 		 Average Reward : 461.63
Episode : 14195 		 Timestep : 102211200 		 Average Reward : 394.26
Episode : 14198 		 Timestep : 102232800 		 Average Reward : 581.36
Episode : 14201 		 Timestep : 102254400 		 Average Reward : 326.7
Episode : 14204 		 Timestep : 102276000 		 Average Reward : 459.88
Episode : 14207 		 Timestep : 102297600 		 Average Reward : 472.43
Episode : 14210 		 Timestep : 102319200 		 Average Reward : 475.12
Episode : 14213 		 Timestep : 102340800 		 Average Reward : 549.74
Episode : 14216 		 Timestep : 102362400 		 Average Reward : 499.

Episode : 14543 		 Timestep : 104716800 		 Average Reward : 498.86
Episode : 14546 		 Timestep : 104738400 		 Average Reward : 452.06
Episode : 14549 		 Timestep : 104760000 		 Average Reward : 477.6
Episode : 14552 		 Timestep : 104781600 		 Average Reward : 511.81
Episode : 14555 		 Timestep : 104803200 		 Average Reward : 528.12
Episode : 14558 		 Timestep : 104824800 		 Average Reward : 499.31
Episode : 14561 		 Timestep : 104846400 		 Average Reward : 312.82
Episode : 14564 		 Timestep : 104868000 		 Average Reward : 517.88
Episode : 14567 		 Timestep : 104889600 		 Average Reward : 614.87
Episode : 14570 		 Timestep : 104911200 		 Average Reward : 444.45
Episode : 14573 		 Timestep : 104932800 		 Average Reward : 387.88
Episode : 14576 		 Timestep : 104954400 		 Average Reward : 468.72
Episode : 14579 		 Timestep : 104976000 		 Average Reward : 585.77
Episode : 14582 		 Timestep : 104997600 		 Average Reward : 383.66
Episode : 14585 		 Timestep : 105019200 		 Average Reward : 423

Episode : 14912 		 Timestep : 107373600 		 Average Reward : 468.36
Episode : 14915 		 Timestep : 107395200 		 Average Reward : 502.79
Episode : 14918 		 Timestep : 107416800 		 Average Reward : 474.25
Episode : 14921 		 Timestep : 107438400 		 Average Reward : 428.11
Episode : 14924 		 Timestep : 107460000 		 Average Reward : 556.91
Episode : 14927 		 Timestep : 107481600 		 Average Reward : 483.55
Episode : 14930 		 Timestep : 107503200 		 Average Reward : 519.78
Episode : 14933 		 Timestep : 107524800 		 Average Reward : 573.21
Episode : 14936 		 Timestep : 107546400 		 Average Reward : 367.71
Episode : 14939 		 Timestep : 107568000 		 Average Reward : 467.88
Episode : 14942 		 Timestep : 107589600 		 Average Reward : 594.14
Episode : 14945 		 Timestep : 107611200 		 Average Reward : 500.35
Episode : 14948 		 Timestep : 107632800 		 Average Reward : 511.81
Episode : 14951 		 Timestep : 107654400 		 Average Reward : 510.38
Episode : 14954 		 Timestep : 107676000 		 Average Reward : 43

Episode : 15269 		 Timestep : 109944000 		 Average Reward : 358.38
Episode : 15272 		 Timestep : 109965600 		 Average Reward : 562.57
Episode : 15275 		 Timestep : 109987200 		 Average Reward : 572.45
Episode : 15278 		 Timestep : 110008800 		 Average Reward : 385.92
Episode : 15281 		 Timestep : 110030400 		 Average Reward : 513.58
Episode : 15284 		 Timestep : 110052000 		 Average Reward : 478.5
Episode : 15287 		 Timestep : 110073600 		 Average Reward : 502.52
Episode : 15290 		 Timestep : 110095200 		 Average Reward : 550.8
Episode : 15293 		 Timestep : 110116800 		 Average Reward : 521.01
Episode : 15296 		 Timestep : 110138400 		 Average Reward : 422.5
Episode : 15299 		 Timestep : 110160000 		 Average Reward : 598.98
Episode : 15302 		 Timestep : 110181600 		 Average Reward : 547.28
Episode : 15305 		 Timestep : 110203200 		 Average Reward : 603.64
Episode : 15308 		 Timestep : 110224800 		 Average Reward : 575.9
Episode : 15311 		 Timestep : 110246400 		 Average Reward : 488.97

Episode : 15638 		 Timestep : 112600800 		 Average Reward : 490.28
Episode : 15641 		 Timestep : 112622400 		 Average Reward : 481.69
Episode : 15644 		 Timestep : 112644000 		 Average Reward : 596.98
Episode : 15647 		 Timestep : 112665600 		 Average Reward : 462.66
Episode : 15650 		 Timestep : 112687200 		 Average Reward : 395.46
Episode : 15653 		 Timestep : 112708800 		 Average Reward : 418.98
Episode : 15656 		 Timestep : 112730400 		 Average Reward : 343.57
Episode : 15659 		 Timestep : 112752000 		 Average Reward : 380.93
Episode : 15662 		 Timestep : 112773600 		 Average Reward : 464.79
Episode : 15665 		 Timestep : 112795200 		 Average Reward : 523.54
Episode : 15668 		 Timestep : 112816800 		 Average Reward : 426.64
Episode : 15671 		 Timestep : 112838400 		 Average Reward : 495.97
Episode : 15674 		 Timestep : 112860000 		 Average Reward : 376.35
Episode : 15677 		 Timestep : 112881600 		 Average Reward : 658.08
Episode : 15680 		 Timestep : 112903200 		 Average Reward : 53

In [ ]:


checkpoint_path = "PPO_pong_game_24000_0.pth"
# checkpoint_path = "./PPO_preTrained/pong_game/PPO_pong_game_1798_0.pth"
ppo_agent.load(checkpoint_path)
env = pongGym.DoublePong()
state = env.reset()+torch.quantization.DeQuantStub()
current_ep_reward = 0
for t in range(1, max_ep_len+1):

    # select action with policy
    action = ppo_agent.select_action(state)
    state, reward, done, _ = env.step(action)
#         if i_episode > 100:
    env.render()
    time.sleep(1/60)
    # saving reward and is_terminals
    ppo_agent.buffer.rewards.append(reward)
    ppo_agent.buffer.is_terminals.append(done)

    time_step +=1
    current_ep_reward += reward


    if done: 
        break




log_f.close()
env.close()


